In [1]:
import polars as pl
import os

pl.Config.set_fmt_str_lengths(200)
pl.Config.set_fmt_float("full")

polars.config.Config

In [2]:
# import file paths
gmx_file = '../data/dune_perp_margin/gmx.csv'
gns_file = '../data/dune_perp_margin/gns.csv'
kwenta_file = '../data/dune_perp_margin/kwenta.csv'
level_file = '../data/dune_perp_margin/level.csv'
mux_file = '../data/dune_perp_margin/mux.csv'
perp_file = '../data/dune_perp_margin/perp.csv'
vertex_file = '../data/dune_perp_margin/vertex.csv'

In [3]:
# load files into dataframes
gmx = pl.read_csv(gmx_file)
gns = pl.read_csv(gns_file)
kwenta = pl.read_csv(kwenta_file)
level = pl.read_csv(level_file)
mux = pl.read_csv(mux_file)
perp = pl.read_csv(perp_file)
vertex = pl.read_csv(vertex_file)

In [4]:
# extract date from time str
gmx = gmx.with_columns(pl.col("time_day").str.split("T").apply(lambda s: s[0]).str.to_date(format='%Y-%m-%d'))

kwenta = kwenta.with_columns(pl.col("time").str.split(' ').apply(lambda s: s[0]).str.to_date(format='%Y-%m-%d'))

level = level.with_columns(pl.col("time").str.split(' ').apply(lambda s: s[0]).str.to_date(format='%Y-%m-%d'))

mux = mux.with_columns(pl.col("time").str.split(' ').apply(lambda s: s[0]).str.to_date(format='%Y-%m-%d'))

perp = perp.with_columns(pl.col("time").str.split(' ').apply(lambda s: s[0]).str.to_date(format='%Y-%m-%d'))

In [5]:
# split gns pair_name token (e.g. "BTC/USD" to BTC) and extract date from time str
gns = gns.with_columns(
    pl.col("pair_name").str.split("/").apply(lambda s: s[0]).alias("token"),
    pl.col("time").str.split(' ').apply(lambda s: s[0]).str.to_date(format='%Y-%m-%d')
)

In [6]:
# filter vertex for type "perp"
vertex = vertex.filter(vertex['type'] == 'Perp')

# split vertex market pair into a token (e.g. "BTC-PERP" to "BTC"). Extract date from time str
vertex = vertex.with_columns(
    pl.col("market").str.split("-").apply(lambda s: s[0]).alias("token"),
    pl.col("date").str.split(' ').apply(lambda s: s[0]).str.to_date(format='%Y-%m-%d')
)

In [7]:
# rename columns
gmx = gmx.select([
    pl.col('time_day').alias('timestamp'),
    pl.col('token').alias('token'),
    pl.col('volume').alias('volume')
])

gns = gns.select([
    pl.col('time').alias('timestamp'),
    pl.col('token').alias('token'),
    pl.col('total_volume').alias('volume')
])

kwenta = kwenta.select([
    pl.col('time').alias('timestamp'),
    pl.col('token').alias('token'),
    pl.col('total_volume').alias('volume')
])

level = level.select([
    pl.col('time').alias('timestamp'),
    pl.col('symbol').alias('token'),
    pl.col('volume').alias('volume')
])

mux = mux.select([
    pl.col('time').alias('timestamp'),
    pl.col('token_name').alias('token'),
    pl.col('total_volume').alias('volume')
])

perp = perp.select([
    pl.col('time').alias('timestamp'),
    pl.col('token').alias('token'),
    pl.col('total_volume').alias('volume')
])

vertex = vertex.select([
    pl.col('date').alias('timestamp'),
    pl.col('token').alias('token'),
    pl.col('daily_vol_by_markets').alias('volume')
])


In [8]:
def add_name_cols(df: pl.DataFrame, protocol: str) -> pl.DataFrame:
    """Add protocol name column to dataframe"""
    # add protocol name column called "prtoocol"
    return df.with_columns(pl.lit(protocol).alias('protocol'))

In [14]:
def weekly_vol(df: pl.DataFrame) -> pl.DataFrame:
    '''
    Returns weekly volume for each blockchain in USD
    '''
    df = df.sort("timestamp", descending=False).groupby_dynamic("timestamp",every='1w').agg(pl.col('volume').sum()).select(
        pl.col("timestamp").dt.truncate("1w"),
        pl.col("volume").alias("weekly_vol"),
        )
    return df

In [15]:
gmx_weekly = weekly_vol(gmx)
gns_weekly = weekly_vol(gns)
kwenta_weekly = weekly_vol(kwenta)
level_weekly = weekly_vol(level)
mux_weekly = weekly_vol(mux)
perp_weekly = weekly_vol(perp)
vertex_weekly = weekly_vol(vertex)

In [16]:
gmx_weekly = add_name_cols(gmx_weekly, 'GMX')
gns_weekly = add_name_cols(gns_weekly, 'GNS')
kwenta_weekly = add_name_cols(kwenta_weekly, 'KWENTA')
level_weekly = add_name_cols(level_weekly, 'LEVEL')
mux_weekly = add_name_cols(mux_weekly, 'MUX')
perp_weekly = add_name_cols(perp_weekly, 'PERP')
vertex_weekly = add_name_cols(vertex_weekly, 'VERTEX')

In [19]:
# concat weekly data
weekly_perp_vol = pl.concat([
    gmx_weekly,
    gns_weekly,
    kwenta_weekly,
    level_weekly,
    mux_weekly,
    perp_weekly,
    vertex_weekly
])

In [20]:
# save to csv in data folder
weekly_perp_vol.write_csv('../data/weekly_perp_vol.csv')

### Daily data with tokens

In [21]:
gmx = add_name_cols(gmx, 'GMX')
gns = add_name_cols(gns, 'GNS')
kwenta = add_name_cols(kwenta, 'KWENTA')
level = add_name_cols(level, 'LEVEL')
mux = add_name_cols(mux, 'MUX')
perp = add_name_cols(perp, 'PERP')
vertex = add_name_cols(vertex, 'VERTEX')

In [22]:
# concat dataframes
perp_margin_vol = pl.concat([gmx, gns, kwenta, level, mux, perp, vertex])

In [23]:
perp_margin_vol.head(5)

timestamp,token,volume,protocol
date,str,f64,str
2023-07-01,"""LINK""",998398.9461109834,"""GMX"""
2023-07-01,"""WETH""",44020376.9708315,"""GMX"""
2023-07-01,"""UNI""",2270084.2464602906,"""GMX"""
2023-07-01,"""WBTC""",8127496.180891069,"""GMX"""
2023-07-01,"""AVAX""",1529145.5261540012,"""GMX"""


In [24]:
# save to csv in data folder
perp_margin_vol.write_csv('../data/perp_margin_vol.csv')